In [34]:
#r "C:\Users\dev\solutions\ds_showdown\ds.Main\bin\Debug\net8.0\ds.Domain.dll"
#r "C:\Users\dev\solutions\ds_showdown\ds.Main\bin\Debug\net8.0\ds.Application.dll"
#r "C:\Users\dev\solutions\ds_showdown\ds.Main\bin\Debug\net8.0\ds.Utility.dll"
#r "C:\Users\dev\solutions\ds_showdown\ds.Main\bin\Debug\net8.0\ds.Main.dll"
#r "C:\Users\dev\solutions\ds_showdown\ds.Main\bin\Debug\net8.0\Newtonsoft.Json.dll"

using ds.Application;
using ds.Main;
using ds.Utility;
using Newtonsoft.Json;
using System.IO;
using System.Collections.Concurrent;
using System.Threading;

Dictionary<string, string> englishDictionary = JsonConvert.DeserializeObject<Dictionary<string, string>>(File.ReadAllText(@"C:\Users\dev\solutions\ds_showdown\ds.Main\dictionary.json")) ?? new Dictionary<string, string>();
englishDictionary.GetIntroduction();

Dictionary<string, string> hashMap = new Dictionary<string, string>();
BinarySearchTree tree = new BinarySearchTree();

var threadIds = new ConcurrentBag<int>();
string wordToSearch = "programme";

Parallel.Invoke(
    () =>
    {
        int threadId = Thread.CurrentThread.ManagedThreadId;
        threadIds.Add(threadId);
        string hashMapInsertionMetrics = Instrumentation.GetMetrics($"HashMap Insertion ({threadId})", () =>
        {
            hashMap.IterateInsert(englishDictionary);
        });
        hashMapInsertionMetrics.DisplayInfo(hm => Console.WriteLine(hm));
    },
    () =>
    {
        int threadId = Thread.CurrentThread.ManagedThreadId;
        threadIds.Add(threadId);
        string treeInsertionMetrics = Instrumentation.GetMetrics($"Binary Search Tree Insertion ({threadId})", () =>
        {
            tree.IterateInsert(englishDictionary);
        });
        treeInsertionMetrics.DisplayInfo(bst => Console.WriteLine(bst));
    }
);

string hMapDefinition = string.Empty, treeDefinition = string.Empty;
Thread.Sleep(1000);
Parallel.Invoke(
    () => { GC.Collect(); Thread.Sleep(1000); },
    () =>
    {
        int threadId = Thread.CurrentThread.ManagedThreadId;
        threadIds.Add(threadId);
        Console.WriteLine();
        string hashMapSearchMetrics = Instrumentation.GetMetrics($"HashMap LookUp ({threadId})", () =>
        {
            hMapDefinition = hashMap[wordToSearch]!;
        });
        hashMapSearchMetrics.DisplayInfo(hm => Console.WriteLine(hm));
    },
    () =>
    {
        int threadId = Thread.CurrentThread.ManagedThreadId;
        threadIds.Add(threadId);
        string treeSearchMetrics = Instrumentation.GetMetrics($"Binary Search Tree LookUp ({threadId})", () =>
        {
            treeDefinition = tree.Search("programme")!;
        });
        treeSearchMetrics.DisplayInfo(bst => Console.WriteLine(bst));
    }
);

Extra.GetOutro(
    $"{wordToSearch} (HMAP Definition): {hMapDefinition}",
    $"{wordToSearch} (TREE Definition): {treeDefinition}"
);

SPECIFICATIONS: 
Total number of words: 102217.
Length of all words combined: 898571.
Length of all definitions combined: 20882783

HashMap Insertion (13) time: 228 milliseconds
HashMap Insertion (13) memory consumption: 5231280 bytes
Binary Search Tree Insertion (26) time: 514 milliseconds
Binary Search Tree Insertion (26) memory consumption: 8324696 bytes

HashMap LookUp (26) time: 177 milliseconds
HashMap LookUp (26) memory consumption: 4136 bytes
Binary Search Tree LookUp (15) time: 278 milliseconds
Binary Search Tree LookUp (15) memory consumption: 32800 bytes

programme (HMAP Definition): That which is written or printed as a public notice or advertisement; a scheme; a prospectus; especially, a brief outline or explanation of the order to be pursued, or the subjects embraced, in any public exercise, performance, or entertainment; a preliminary sketch. Programme music (Mus.), descriptive instrumental music which requires an argument or programme to explain the meaning of its sever